In [1]:
from train import TransdimensionalFlowModule
from sampling import any_order_mask_insertion_euler_sampling, SamplingTraceDatapoint
import os

In [24]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [12]:
# Brian's model
# checkpoint_path = "./checkpoints/bracket/any-order/linear-Brian/epoch=89-val_loss=8.7507.ckpt"

# Jay's model
checkpoint_path = "./checkpoints/bracket/any-order/linear-Jay/epoch=39-val_loss=6.4387.ckpt"

model = TransdimensionalFlowModule.load_from_checkpoint(checkpoint_path)

In [13]:
steps = 2000
batch_size = 20
samples, trace = any_order_mask_insertion_euler_sampling(
    model,
    model.interpolant,
    steps=steps,
    mask=0,
    pad=3,
    batch_size=batch_size,
    max_length=64,
    return_trace=True,
)

step 0
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
step 51
step 52
step 53
step 54
step 55
step 56
step 57
step 58
step 59
step 60
step 61
step 62
step 63
step 64
step 65
step 66
step 67
step 68
step 69
step 70
step 71
step 72
step 73
step 74
step 75
step 76
step 77
step 78
step 79
step 80
step 81
step 82
step 83
step 84
step 85
step 86
step 87
step 88
step 89
step 90
step 91
step 92
step 93
step 94
step 95
step 96
step 97
step 98
step 99
step 100
step 101
step 102
step 103
step 104
step 105
step 106
step 107
step 108
step 109
step 110
step 111
step 112
step 113
step 114
step 115
step 116
step 117
step 118
step 119
step 120
step 121
step 122
ste

In [14]:
def process_trace(trace: list[SamplingTraceDatapoint]):
    event_type_mapping = dict(change="c", insertion="i")
    token_mapping = {0: "m", 1: "(", 2: ")"}

    def _process_datapoint(datapoint: SamplingTraceDatapoint):
        return dict(
            t=datapoint.t,
            a=event_type_mapping[datapoint.event_type],
            tk=token_mapping[datapoint.token],
            i=datapoint.position,
        )

    return [_process_datapoint(datapoint) for datapoint in trace]

In [15]:
samples

tensor([[3, 3, 3,  ..., 3, 3, 3],
        [1, 1, 2,  ..., 3, 3, 3],
        [3, 3, 3,  ..., 3, 3, 3],
        ...,
        [3, 3, 3,  ..., 3, 3, 3],
        [1, 1, 1,  ..., 3, 3, 3],
        [1, 2, 3,  ..., 3, 3, 3]], device='cuda:0')

In [34]:
from IPython.display import display, HTML
import json

# Select the index of the tensor to visualize
id = 19
assert 0 <= id < batch_size, "id must be in [0, batch_size)"

events = process_trace(trace[id])
print(events)

display(
    HTML(f"""
<style>
.token {{
    display: inline-block;
    padding: 5px 10px;
    margin: 2px;
    border: 1px solid #aaa;
    border-radius: 4px;
    font-family: monospace;
    transition: background 0.5s;
}}
.hi-i {{ background: lightgreen; }}
.hi-c {{ background: yellow; }}
#s {{ min-height: 30px; margin-bottom: 20px; }}
</style>
<div id="s"></div>
<button id="r">Replay</button>
<script>
var ev = {json.dumps(events)};
var s = [];
var delay = 100; // Delay between steps (ms)

function render(hIdx, action) {{
    var container = document.getElementById("s");
    container.innerHTML = "";
    s.forEach(function(token, idx) {{
        var span = document.createElement("span");
        span.className = "token" + (idx === hIdx ? " hi-" + action : "");
        span.textContent = token;
        container.appendChild(span);
    }});
}}

async function run() {{
    s = [];
    render();
    
    for (const e of ev) {{
        if (e.a === "i") {{
            // Insert token at position e.i
            var pos = (e.i < 0 || e.i > s.length) ? Math.floor(s.length / 2) : e.i;
            s.splice(pos, 0, e.tk);
            render(pos, "i");
        }} else {{
            // Change token at position e.i
            var pos = (e.i === undefined || e.i < 0 || e.i >= s.length) ? Math.floor(s.length / 2) : e.i;
            s[pos] = e.tk;
            render(pos, "c");
        }}
        
        // Small delay to make the animation visible (but still synchronous in execution order)
        await new Promise(resolve => setTimeout(resolve, delay));
    }}
}}

document.getElementById("r").onclick = run;
run(); // Start automatically
</script>
""")
)

KeyError: 3